<a href="https://colab.research.google.com/github/urieliram/Pyomo_example/blob/main/UCP_Pyomo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
# !pip install -q pyomo # instalación del lenguaje algebraico pyomo
# !apt-get install -y -qq coinor-cbc #instalación solver cbc
# #!apt-get install -y -qq glpk-utils #instalación solver glpk

In [37]:
from pyomo.environ import Suffix, value
from pyomo.opt import SolverStatus, TerminationCondition, SolverFactory

# Ejemplo de uso de Pyomo para resolver un problema de asignación de unidades

**Conjuntos**
-  $T$ : Conjunto de intervalos de tiempo.
-  $J$ : Conjunto de unidades generadoras.

**Parámetros**
- $c_j$ : Costo de operación de la unidad generadora \( $j$ \).
-  $c_j^U$ : Costo de  arranque de la unidad generadora \( $j$ \).
-  $De_t$ : Demanda en el intervalo de tiempo \( $t$ \).
-  $R_t$ : Reserva rodante requerida en el intervalo de tiempo \( $t$ \).
-  $R_j^U$ : Límite superior de rampa para la unidad generadora \( $j$ \).
-  $S_j^U$ : Límite superior de rampa de arranque para la unidad generadora \( $j$ \).
-  $R_j^D$ : Límite inferior de rampa para la unidad generadora \( $j$ \).
-  $S_j^D$ : Límite inferior de rampa de apagado para la unidad generadora \( $j$ \).
- $T_j^U $: Tiempo mínimo encendido para la unidad generadora \( $j$ \) (para $y_{jt}$).
-  $T_j^D$: Tiempo mínimo apagado para la unidad generadora \( $j$ \) (para $z_{jt}$).
-  $P_j^\text{min}$ : Capacidad mínima de generación para la unidad generadora \( $j$ \).
-  $P_j^\text{max}$ : Capacidad máxima de generación para la unidad generadora \( $j$ \).
-  $U_j$ : Periodos encendido la unidad generadora \( $j$ \).
-  $D_j$ : Periodo apagado para la unidad generadora \( $j$ \).

**Variables de decisión**
-  $p_{jt}$ : Generación de la unidad \( $j$ \) en el intervalo \( $t$ \).
-  $\bar{p}_{jt}$ : Generación adicional de la unidad \( $j$ \) en el intervalo \( $t$ \).
-  $v_{jt}$ : Variable binaria que indica si la unidad generadora \( $j$ \) está en operación en el intervalo \( $t$ \).
-  $y_{jt}$ : Variable binaria que indica si se arranca la unidad generadora \( $j$ \) en el intervalo \( $t$ \).
-  $z_{jt}$ : Variable binaria que indica si se apaga la unidad generadora \( $j$ \) en el intervalo \( $t$ \).

**Modelo**
\begin{align*}
\text{Min} \quad & \sum_{t\in T} \sum_{j\in J} ( c_{j} p_{jt} + c_{j}^\text{U} y_{jt}) \\
\text{Sujeto a:} \quad & \sum_{j\in J} p_{jt} = De_{t}, & t \in T \\
& \sum_{j\in J} \bar{p}_{jt} \geq De_{t} + R_{t}, &  t \in T\\
& v_{j,t-1} - v_{jt} + y_{jt} - z_{jt} = 0 & j\in J, t \in T  \\
& p_{jt} - p_{j,t-1} \leq R_{j}^\text{U} v_{j,t-1} + S_{j}^\text{U} y_{jt} & j\in J, t \in T  \\
& p_{j,t-1} - p_{jt} \leq R_{j}^\text{D} v_{j,t} + S_{j}^\text{D} z_{jt} & j\in J, t \in T  \\
& \sum_{k=t-T_{j}^{U}+1,k \geq 1}^{t} y_{jk} \leq v_{jt} & j\in J, t\in \{L_{j}+1,...,|T|\} \\
& v_{jt}+\sum_{k=t-T_{j}^{D}+1,k \geq 1}^{t} z_{jk} \leq 1 & j\in J, t\in \{F_{j}+1,...,|T|\} \\
& P_{j}^{\text{min}} v_{jt} \leq p_{jt} \leq \bar{p}_{jt} \leq P_{j}^{\text{max}} v_{jt} & j\in J, t \in T  \\
& \bar{p}_{jt} \leq p_{j,t-1} + R_{j}^{\text{U}} v_{j,t-1} + S_{j}^{\text{U}} y_{jt} & j\in J, t \in T  \\
& \bar{p}_{jt} \leq P_{j}^{\text{max}} (v_{jt}-z_{j,t+1})+ z_{j,t+1} S_{j}^{\text{D}} & j\in J, t =\{1,2,...,|T|-1\} \\
& p_{jt}, \bar{p}_{jt} \geq 0 \\
& v_{jt}, y_{jt}, z_{jt} \in \{0,1\} \\
 \quad \text{donde:}  \\
& L_{j} = min(U_{j},|T|) \\
& F_{j} = min(D_{j},|T|)
\end{align*}

Modelo extraido de: M. F. Anjos and A. J. Conejo., Unit Commitment in Electric Energy Systems. Foundations and Trends in Electric Energy Systems. Vol. 1, No. 4, 220--310, 2017.

In [38]:
#Conjuntos
T    = {1,2,3,4,5,6}
G    = {1,2,3}

# Parámetros
De   = {1: 240, 2: 250, 3: 200, 4: 170, 5: 230, 6: 190} # demanda
R    = {1: 0, 2: 10, 3: 10, 4: 10, 5: 10, 6: 10} #reserva

# limites potencia
Pmax = {1: 300, 2: 200, 3: 100}
Pmin = {1: 80, 2: 50, 3: 30}

# costos
cU   = {1: 800, 2: 500, 3: 250}
c    = {1: 5, 2: 15, 3: 30}

# rampas
RD   = {1: 30, 2: 40, 3: 50}
RU   = {1: 50, 2: 60, 3: 70}
SD   = {1: 80, 2: 50, 3: 30}
SU   = {1: 100, 2: 70, 3: 40}

# Periodos mínimos de encendido y de apagado.
TU   = {1: 3, 2: 2, 3: 1}
TD   = {1: 2, 2: 2, 3: 2}

# Estados iniciales de las unidades generadoras
U    = {1: 2, 2: 0, 3: 0} # tiempo que ha estado prendida
D    = {1: 0, 2: 0, 3: 0} # tiempo que ha estado apagado

v0   = {1: 1, 2: 0, 3: 0}
p0   = {1: 120, 2: 0, 3: 0}

In [39]:
# importaciones de librerias
from pyomo.environ import *

# Aqui se define el modelo
def construir_modelo(LP = False, commit = None):

    # Crear un modelo
    model      = ConcreteModel()

    # Conjuntos
    model.G    = Set(initialize = G)
    model.T    = Set(initialize = T)

    # Parametros
    model.De    = Param(model.T , initialize = De   , within = Any)
    model.R     = Param(model.T , initialize = R    , within = Any)
    model.Pmax  = Param(model.G , initialize = Pmax , within = Any)
    model.Pmin  = Param(model.G , initialize = Pmin , within = Any)
    model.c     = Param(model.G , initialize = c    , within = Any)
    model.cU    = Param(model.G , initialize = cU   , within = Any)
    model.RU    = Param(model.G , initialize = RU   , within = Any)
    model.RD    = Param(model.G , initialize = RD   , within = Any)
    model.SU    = Param(model.G , initialize = SU   , within = Any)
    model.SD    = Param(model.G , initialize = SD   , within = Any)
    model.U     = Param(model.G , initialize = U    , within = Any)
    model.D     = Param(model.G , initialize = D    , within = Any)
    model.TU    = Param(model.G , initialize = TU   , within = Any)
    model.TD    = Param(model.G , initialize = TD   , within = Any)
    model.v0    = Param(model.G , initialize = v0   , within = Any)
    model.p0    = Param(model.G , initialize = p0   , within = Any)

    # Variables
    model.p     = Var( model.G , model.T , bounds = (0.0,99999.0))
    model.pb    = Var( model.G , model.T , bounds = (0.0,99999.0))

    # Si se quiere resolver en continuas
    if LP == True:
        model.v     = Var( model.G , model.T , within = UnitInterval) # continuo entre cero y uno
        model.y     = Var( model.G , model.T , within = UnitInterval)
        model.z     = Var( model.G , model.T , within = UnitInterval)
    else:
        model.v     = Var( model.G , model.T , within = Binary)
        model.y     = Var( model.G , model.T , within = Binary)
        model.z     = Var( model.G , model.T , within = Binary)

    def funcion_objetivo(m):
        return sum(( m.c[j] * m.p[j,t] + m.cU[j] * m.y[j,t] ) for t in m.T for j in m.G)
    model.obj = Objective(rule = funcion_objetivo, sense=minimize)

    def demanda(m,t):
        return sum( m.p[j,t] for j in m.G ) == m.De[t]
    model.demanda = Constraint(model.T, rule = demanda)

    def demanda_reserva(m,t):
        return sum( m.pb[j,t] for j in m.G ) >= m.De[t] + m.R[t]
    model.demanda_reserva = Constraint(model.T, rule = demanda_reserva)

    def logical_rule(m,j,t):
        if t == 1:
            return m.v0[j]    - m.v[j,t] + m.y[j,t] - m.z[j,t] == 0
        else:
            return m.v[j,t-1] - m.v[j,t] + m.y[j,t] - m.z[j,t] == 0
    model.logical_rule = Constraint(model.G,model.T, rule = logical_rule)

    def rampas_subida(m,j,t):
        if t == 1:
            return m.p[j,t] - m.p0[j]    <= m.RU[j] * m.v0[j]    + m.SU[j] * m.y[j,t]
        else:
            return m.p[j,t] - m.p[j,t-1] <= m.RU[j] * m.v[j,t-1] + m.SU[j] * m.y[j,t]
    model.rampas_subida = Constraint(model.G,model.T, rule = rampas_subida)

    def rampas_bajada(m,j,t):
        if t == 1:
            return m.p0[j]    - m.p[j,t] <= m.RD[j] * m.v[j,t] + m.SD[j] * m.z[j,t]
        else:
            return m.p[j,t-1] - m.p[j,t] <= m.RD[j] * m.v[j,t] + m.SD[j] * m.z[j,t]
    model.rampas_bajada = Constraint(model.G,model.T,rule = rampas_bajada)

    def tiempo_min_encendido(m,j,t):
        Lj = min( m.U[j],len(m.T) )
        if t >= Lj + 1:
            return sum( m.y[j,k] for k in range(t-m.TU[j]+1,t) if k>=1) <= m.v[ j , t ]
        else:
            return Constraint.Skip
    model.tiempo_min_encendido = Constraint(model.G, model.T,rule = tiempo_min_encendido)

    def tiempo_min_apagado(m,j,t):
        Fj = min( m.D[j],len(m.T) )
        if t >= Fj + 1:
            return m.v[j,t] + sum( m.z[j,k] for k in range(t-m.TD[j]+1,t) if k>=1) <= 1
        else:
            return Constraint.Skip
    model.tiempo_min_apagado = Constraint(model.G,model.T,rule = tiempo_min_apagado)

    def limite_gen_uno(m,j,t):
        return m.Pmin[j] * m.v[j,t] <= m.p[j,t]
    model.limite_gen_uno = Constraint(model.G,model.T,rule = limite_gen_uno)

    def limite_gen_dos(m,j,t):
        return m.p[j,t] <= m.pb[j,t]
    model.limite_gen_dos = Constraint(model.G,model.T,rule = limite_gen_dos)

    def Limite_gen_tres(m,j,t):
        return m.pb[j,t] <= m.Pmax[j] * m.v[j,t]
    model.Limite_gen_tres = Constraint(model.G,model.T,rule = Limite_gen_tres)

    def limite_arranque(m,j,t):
        if t == 1:
            return m.pb[j,t] <= m.p0[j]    + m.RU[j] * m.v0[j]    + m.SU[j] * m.y[j,t]
        else:
            return m.pb[j,t] <= m.p[j,t-1] + m.RU[j] * m.v[j,t-1] + m.SU[j] * m.y[j,t]
    model.limite_arranque = Constraint(model.G,model.T,rule = limite_arranque)

    def limite_apagado(m,j,t):
        if t == len(m.T):
            return Constraint.Skip
        else:
            return m.pb[j,t] <= m.Pmax[j] * ( m.v[j,t] - m.z[j,t+1] ) + m.z[j,t+1] * m.SD[j]
    model.limite_apagado = Constraint(model.G,model.T,rule = limite_apagado)

# recibe el resultado de la varioable de optimizacion v
    if commit:
      print("FIJANDO SOLUCIÓN ENTERA")
      for key, value in commit.items():
        g, t = key
        model.v[g, t].fix(value) ## Hard fixing

    return model

# Solucionando el modelo

## Escribiendo el problema a formatos estandar 

In [50]:
model = construir_modelo()
SolverFactory('cplex', executable='C:/ILOG/CPLEX_Studio129/cplex/bin/x64_win64/cplex').solve(model).write()

model.pprint()
print('\n z = ', model.obj())

    (type: set).  This WILL potentially lead to nondeterministic behavior in
    Pyomo
    (type: set).  This WILL potentially lead to nondeterministic behavior in
    Pyomo
# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: tmpw0hei0eu
  Lower bound: 9099.999999999998
  Upper bound: 9099.999999999998
  Number of objectives: 1
  Number of constraints: 188
  Number of variables: 91
  Number of nonzeros: 522
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: 0.09
  Termination condition: optimal
  Termination message: MIP - Integer optimal solution\x3a 

In [41]:
def imprime_tabla():
  from prettytable import PrettyTable
  # Crear una tabla
  tabla = PrettyTable()
  tabla.field_names = ['G', 'T', 'p', 'pb', 'v', 'y', 'z']

  # Llenar la tabla con los valores de las variables
  for g in G:
      for t in T:
          tabla.add_row([g, t, model.p[g, t].value, model.pb[g, t].value,
                        model.v[g, t].value, model.y[g, t].value, model.z[g, t].value])
  # Imprimir la tabla
  print(tabla)
imprime_tabla()

+---+---+-------------------------+-------------------------+------+------+------+
| G | T |            p            |            pb           |  v   |  y   |  z   |
+---+---+-------------------------+-------------------------+------+------+------+
| 1 | 1 |          170.0          |          170.0          | 1.0  | 0.0  | 0.0  |
| 1 | 2 |    200.00000000000006   |          220.0          | 1.0  | 0.0  | 0.0  |
| 1 | 3 |          200.0          |          210.0          | 1.0  | 0.0  | 0.0  |
| 1 | 4 |          170.0          |          180.0          | 1.0  | -0.0 | 0.0  |
| 1 | 5 |    200.0000000000001    |    210.0000000000001    | 1.0  | -0.0 | -0.0 |
| 1 | 6 |          190.0          |          200.0          | 1.0  | -0.0 | 0.0  |
| 2 | 1 |           70.0          |    70.00000000000006    | 1.0  | 1.0  | 0.0  |
| 2 | 2 |    49.99999999999994    |    49.99999999999994    | 1.0  | 0.0  | 0.0  |
| 2 | 3 |           0.0           |           0.0           | 0.0  | -0.0 | 1.0  |
| 2 

In [42]:
## Escribir el modelo en formato LP
model.write("problema.lp", io_options={'symbolic_solver_labels':True})

## Escribir el modelo en formato MPS
model.write("problema.mps" , io_options = {'symbolic_solver_labels':True})


('problema.mps', 2484151819072)

## Fijando una solución y resolviendo en relajación continua

In [43]:
# Llenamos el diccionario 'commit' con los valores de las variables de asignación
commit = {}
for g in G:
    for t in T:
        commit[(g, t)] = model.v[g, t].value

In [44]:
model = construir_modelo(LP=True,commit=commit)
# Create a 'rc' suffix component on the instance so the solver plugin will know which suffixes to collect
model.rc   = Suffix(direction=Suffix.IMPORT,datatype=Suffix.FLOAT)
model.dual = Suffix(direction=Suffix.IMPORT,datatype=Suffix.FLOAT)

    (type: set).  This WILL potentially lead to nondeterministic behavior in
    Pyomo
    (type: set).  This WILL potentially lead to nondeterministic behavior in
    Pyomo
FIJANDO SOLUCIÓN ENTERA


In [45]:
SolverFactory('cplex', executable='C:/ILOG/CPLEX_Studio129/cplex/bin/x64_win64/cplex').solve(model).write()
print('\n z = ', model.obj())

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: tmpkcvd_li0
  Lower bound: 9100.0
  Upper bound: 9100.0
  Number of objectives: 1
  Number of constraints: 188
  Number of variables: 73
  Number of nonzeros: 356
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: 0.02
  Termination condition: optimal
  Termination message: Dual simplex - Optimal\x3a Objective = 9.1000000000e+03
  Error rc: 0
  Time: 0.09832930564880371
# ----------------------------------------------------------
#   Solution Information
# ---------------------------------------------

In [46]:
model.rc.pprint()

rc : Direction=Suffix.IMPORT, Datatype=Suffix.FLOAT
    Key     : Value
     p[1,1] :   0.0
     p[1,2] :   0.0
     p[1,3] :   0.0
     p[1,4] :   0.0
     p[1,5] :   0.0
     p[1,6] :   0.0
     p[2,1] :   0.0
     p[2,2] :   0.0
     p[2,3] :  10.0
     p[2,4] :  10.0
     p[2,5] :  10.0
     p[2,6] :  10.0
     p[3,1] :  15.0
     p[3,2] :  25.0
     p[3,3] :  25.0
     p[3,4] :  25.0
     p[3,5] :   0.0
     p[3,6] :  25.0
    pb[1,1] :   0.0
    pb[1,2] :   0.0
    pb[1,3] :   0.0
    pb[1,4] :   0.0
    pb[1,5] :   0.0
    pb[1,6] :   0.0
    pb[2,1] :   0.0
    pb[2,2] :   0.0
    pb[2,3] :   0.0
    pb[2,4] :   0.0
    pb[2,5] :   0.0
    pb[2,6] :   0.0
    pb[3,1] :   0.0
    pb[3,2] :   0.0
    pb[3,3] :   0.0
    pb[3,4] :   0.0
    pb[3,5] :   0.0
    pb[3,6] :   0.0
     y[1,1] :   0.0
     y[1,2] : 800.0
     y[1,3] : 800.0
     y[1,4] : 800.0
     y[1,5] : 800.0
     y[1,6] :   0.0
     y[2,1] :   0.0
     y[2,2] : 500.0
     y[2,3] : 500.0
     y[2,4] : 500.0
     y[2

In [47]:
for index in model.p:
    print(model.rc[model.p[index[0],index[1]]])

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
10.0
10.0
10.0
10.0
15.0
25.0
25.0
25.0
0.0
25.0


## Cálculo de las variables duales

In [48]:
model.dual.pprint()

dual : Direction=Suffix.IMPORT, Datatype=Suffix.FLOAT
    Key                       : Value
         Limite_gen_tres[1,1] :    0.0
         Limite_gen_tres[1,2] :    0.0
         Limite_gen_tres[1,3] :    0.0
         Limite_gen_tres[1,4] :    0.0
         Limite_gen_tres[1,5] :    0.0
         Limite_gen_tres[1,6] :    0.0
         Limite_gen_tres[2,1] :    0.0
         Limite_gen_tres[2,2] :    0.0
         Limite_gen_tres[2,3] :    0.0
         Limite_gen_tres[2,4] :    0.0
         Limite_gen_tres[2,5] :    0.0
         Limite_gen_tres[2,6] :    0.0
         Limite_gen_tres[3,1] :    0.0
         Limite_gen_tres[3,2] :    0.0
         Limite_gen_tres[3,3] :    0.0
         Limite_gen_tres[3,4] :    0.0
         Limite_gen_tres[3,5] :    0.0
         Limite_gen_tres[3,6] :    0.0
                   demanda[1] :   15.0
                   demanda[2] :    5.0
                   demanda[3] :    5.0
                   demanda[4] :    5.0
                   demanda[5] :    5.0
           

In [49]:
for t in model.T:
    print(model.dual[ model.demanda[t] ])

15.0
5.0
5.0
5.0
5.0
5.0
